In [1]:
# import libraries
import datetime as dt
import numpy as np 
import pandas as pd 
import plotly as py
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [2]:
# import real-time data
url = 'https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv'
df = pd.read_csv(url, error_bad_lines=False)
print(df.head(3))

         date       state  fips  cases  deaths
0  2020-01-21  Washington    53      1       0
1  2020-01-22  Washington    53      1       0
2  2020-01-23  Washington    53      1       0


In [3]:
# rename columns
df_states = df.groupby(['state', 'date']).sum().reset_index().sort_values('date', ascending=False)
# remove duplicates
df_states = df_states.drop_duplicates(subset = ['state'])
# only consider 50 US states, not territories
notstates = ['Guam','Puerto Rico','Northern Mariana Islands','Virgin Islands']
df_states = df_states[~df_states['state'].isin(notstates)]

In [4]:
# convert state names to 2-letter code
state_codes = {
    'District of Columbia' : 'dc','Mississippi': 'MS', 'Oklahoma': 'OK', 
    'Delaware': 'DE', 'Minnesota': 'MN', 'Illinois': 'IL', 'Arkansas': 'AR', 
    'New Mexico': 'NM', 'Indiana': 'IN', 'Maryland': 'MD', 'Louisiana': 'LA', 
    'Idaho': 'ID', 'Wyoming': 'WY', 'Tennessee': 'TN', 'Arizona': 'AZ', 
    'Iowa': 'IA', 'Michigan': 'MI', 'Kansas': 'KS', 'Utah': 'UT', 
    'Virginia': 'VA', 'Oregon': 'OR', 'Connecticut': 'CT', 'Montana': 'MT', 
    'California': 'CA', 'Massachusetts': 'MA', 'West Virginia': 'WV', 
    'South Carolina': 'SC', 'New Hampshire': 'NH', 'Wisconsin': 'WI',
    'Vermont': 'VT', 'Georgia': 'GA', 'North Dakota': 'ND', 
    'Pennsylvania': 'PA', 'Florida': 'FL', 'Alaska': 'AK', 'Kentucky': 'KY', 
    'Hawaii': 'HI', 'Nebraska': 'NE', 'Missouri': 'MO', 'Ohio': 'OH', 
    'Alabama': 'AL', 'Rhode Island': 'RI', 'South Dakota': 'SD', 
    'Colorado': 'CO', 'New Jersey': 'NJ', 'Washington': 'WA', 
    'North Carolina': 'NC', 'New York': 'NY', 'Texas': 'TX', 
    'Nevada': 'NV', 'Maine': 'ME'}

df_states['state_code'] = df_states['state'].apply(lambda x : state_codes[x])

In [5]:
# create static choropleth map
fig = go.Figure(
    data = go.Choropleth(
    locations = df_states['state_code'],
    locationmode = 'USA-states',
    z = df_states['cases'],
    colorscale = 'Reds',
    autocolorscale = False,)
)

today = dt.date.today()
maptitle = 'Confirmed Cases in the US as of '+ today.strftime("%B %d, %Y")

fig.update_layout(
    title_text = maptitle,
    title_x = 0.5,
    geo = dict(
        scope = 'usa',
        projection = go.layout.geo.Projection(type = 'albers usa'),
        showlakes = False),
)

fig.show()

In [6]:
# re-arrange data
df_statedate = df.groupby(['date','state']).sum().reset_index()
notstates = ['Guam','Puerto Rico','Northern Mariana Islands','Virgin Islands']
df_statedate = df_statedate[~df_statedate['state'].isin(notstates)]
df_statedate['state_code'] = df_statedate['state'].apply(lambda x : state_codes[x])

In [7]:
# create dynamic choropleth map
fig = px.choropleth(
        df_statedate, 
        locations='state_code', 
        locationmode = 'USA-states',
        color = 'cases', 
        hover_name = 'state_code', 
        animation_frame = 'date',
        color_continuous_scale = 'Reds'
)

fig.update_layout(
    title_text = 'Spread of Coronavirus in the US',
    title_x = 0.5,
    geo=dict(
        scope = 'usa',
        projection = go.layout.geo.Projection(type = 'albers usa'),
        showlakes = False),
)
    
fig.show()